# Concave Hull
Implementation based on https://github.com/sebastianbeyer/concavehull

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [ ]:
cd '/content/drive/My Drive/masks'

/content/drive/My Drive/masks


In [ ]:
mask_dir = r'/content/drive/My Drive/masks/resultsproc_all_masks/maskscroped/' #directory with the binary masks
save_dir = r'/content/drive/My Drive/masks/resultsproc_all_masks/chull/' #directory where the concave hull regions will be saved

In [ ]:
import os
import tifffile
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)
from scipy import ndimage
from ConcaveHull import concaveHull
from PIL import Image, ImageDraw

#resized size
rx = 100
ry = 100

for msk in os.listdir(mask_dir):
    mask = tifffile.imread(os.path.join(mask_dir, msk))
    x,y = np.shape(mask)
    mask = cv2.resize(mask, (ry,rx)) #resize the mask to decrease computational cost
    mask[mask!=0] = 255.0
    mask_edges = mask - (ndimage.morphology.binary_erosion(mask)*255.0) #obtain the edges to compute concave hull

    #get the coordinates (x,y) of the points belonging to the edges of the mask
    rows, cols = np.where(mask_edges == 255)
    cols = np.expand_dims(cols, axis=-1)
    rows = np.expand_dims(rows, axis=-1)
    points_2d = np.concatenate((cols, rows), axis=-1)

    #compute the concaveHull
    hull = concaveHull(points_2d,16)   #https://github.com/sebastianbeyer/concavehull

    #convert the points into a binary mask (chull)
    polygon = []
    for i in range(np.shape(hull)[0]):
        polygon.append(hull[i][0])
        polygon.append(hull[i][1])

    img = Image.new('L', (ry,rx), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    chull = np.array(img)

    #resize to the original size
    chull = cv2.resize(chull,(y,x))
    plt.figure()
    plt.imshow(chull, cmap='gray')

    #save the concave hull
    chull = (chull*255.0).astype('uint8')
    tifffile.imwrite(os.path.join(save_dir, msk), chull)